In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [4]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

In [5]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [7]:
class Rule:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

    def match(self, example):
        val = example[self.column]
        try:
            if is_numeric(val):
                return val >= self.value
            else:
                return val == self.value
        except TypeError:
            return 'nan'


In [8]:
def build_stump(rows, rule):
    true_rows, false_rows = [], []
    for row in rows:
        if isinstance(row[rule.column],float) and math.isnan(row[rule.column]):
            continue
        if rule.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [9]:
def find_best_split(rows):
    best_gain = 0
    best_rule = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            rule = Rule(col, val)
            true_rows, false_rows = build_stump(rows, rule)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_rule = gain, rule

    return best_gain, best_rule

In [10]:
class Leaf:
    def __init__(self, rows):
        val = class_counts(rows)
        itemMax = max(val.items(), key=lambda x : x[1])
        self.predictions = itemMax[0]

In [11]:
class Decision_Node:
    def __init__(self,rule,true_branch,false_branch):
        self.rule = rule
        self.true_branch = true_branch
        self.false_branch = false_branch

In [12]:
def build_tree(rows, max_depth = 10, depth = 0):
    gain, rule = find_best_split(rows)
    if gain == 0 or depth >= max_depth:
        return Leaf(rows)
    true_rows, false_rows = build_stump(rows, rule)
    true_branch = build_tree(true_rows, max_depth, depth+1)
    false_branch = build_tree(false_rows, max_depth, depth+1)
    return Decision_Node(rule, true_branch, false_branch)

In [14]:
def predict(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.rule.match(row):
        return predict(row, node.true_branch)
    else:
        return predict(row, node.false_branch)

In [13]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "└── Predict", node.predictions)
        return
    print (spacing + str(node.rule))

    print (spacing + '├── True:')
    print_tree(node.true_branch, spacing + "|  ")

    print (spacing + '├── False:')
    print_tree(node.false_branch, spacing + "|  ")

In [ ]:
#testing only 

df = pd.read_csv("Datasets/working_csv7.csv")
Y = df["P. Habitable Class"]
X = df.drop("P. Habitable Class",axis=1)
header = list(df.columns)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

X_train_list = X_train.values.tolist()
X_test_list = X_test.values.tolist()
Y_train_list = Y_train.values.tolist()
Y_test_list = Y_test.values.tolist()
    
training_data = list(map(lambda x:x[0]+[x[1]],zip(X_train_list,Y_train_list)))
testing_data = list(map(lambda x:x[0]+[x[1]],zip(X_test_list,Y_test_list)))

my_tree = build_tree(training_data,3)

y_pred = []
y_test = []

for i,row in enumerate(testing_data):
#     print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(predict(row, my_tree))))
    val = predict(row,my_tree)
    
    y_test.append(row[-1])
#     y_pred.append(list(val.keys())[0])
    y_pred.append(val)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [26]:
#evaluate model on all 500 datasets

from sklearn.model_selection import train_test_split
from functools import reduce

pred_labels = []
actual_labels = []

for i in range(500):
    df = pd.read_csv("Datasets/working_csv"+str(i)+".csv")
    Y = df["P. Habitable Class"]
    X = df.drop("P. Habitable Class",axis=1)

    header = list(df.columns)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

    X_train_list = X_train.values.tolist()
    X_test_list = X_test.values.tolist()
    Y_train_list = Y_train.values.tolist()
    Y_test_list = Y_test.values.tolist()

    training_data = list(map(lambda x:x[0]+[x[1]],zip(X_train_list,Y_train_list)))
    testing_data = list(map(lambda x:x[0]+[x[1]],zip(X_test_list,Y_test_list)))
    
    my_tree = build_tree(training_data)
    
    actual_labels += Y_test_list
    
    for row in testing_data:
        pred_labels.append(predict(row,my_tree))

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(actual_labels, pred_labels)
cm_normalized = cm.astype('float') * 100/ cm.sum(axis=1)[:, np.newaxis]

print(np.around(cm_normalized,2))
print(classification_report(actual_labels, pred_labels))

[[92.47  0.53  7.  ]
 [ 0.92 95.73  3.34]
 [ 9.08  5.01 85.9 ]]
               precision    recall  f1-score   support

   mesoplanet       0.90      0.92      0.91      2815
non-habitable       0.94      0.96      0.95      2812
psychroplanet       0.89      0.86      0.88      2873

     accuracy                           0.91      8500
    macro avg       0.91      0.91      0.91      8500
 weighted avg       0.91      0.91      0.91      8500



In [25]:
print_tree(my_tree)

Is P. HZD >= -0.42?
├── True:
|  └── Predict psychroplanet
├── False:
|  Is P. Habitable >= 1?
|  ├── True:
|  |  └── Predict mesoplanet
|  ├── False:
|  |  └── Predict non-habitable
